Описание workflow:

Поскольку можно делать как базову часть, так и продвинутую (по желанию), я выбрала задания, которые мне было интересно выполнять (но не меньше того количества, которое предполагает базовая часть).

Надеюсь, такой подход отвечает требованиям к выполнению Лабораторной работы.

Спасибо :)

In [1]:
import os
import re
import json
import requests
import pandas as pd

In [2]:
import logging

In [3]:
SOURCES_PATH = './sources/'
RESULTS_PATH = './tmp/'

# Base tasks

## Task 1. Анализ данных из текстового файла

In [4]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename=RESULTS_PATH + 'task1.log',
    level=logging.INFO,
    format='%(levelname)s: %(message)s'
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/polina/Desktop/SKMaga/pass_hometasks/python_programming/practica#1/tmp/task1.log'

In [5]:
def read_to_dict(
    filepath: str=SOURCES_PATH + 'data.txt'
):
    '''
        total_sum:
        products: {
            {
                name: {
                    total_amount:
                    total_sum: 
                }
            }
        }
    '''

    reault = {
        'total_sum': 0,
        'products': {}
    }

    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            line  = f.readline()
            
            while line:
                name, amount, price = line.split(',')
                amount.strip()
                price.strip()

                amount = int(amount)
                price = float(price)
                
                if reault['products'].get(name):
                    reault['products'][name]['total_amount'] += amount
                    reault['products'][name]['total_sum'] += price
                else:
                    reault['products'][name] = {
                        'total_amount': amount,
                        'total_sum': price
                    }

                reault['total_sum'] += price

                line  = f.readline()

        logging.info(f'Файл {filepath} прочитан успешно')

    except (FileNotFoundError, PermissionError, IOError) as e:
        logging.error(f'Ошибка чтения файла {filepath}: {e}')

    return reault

In [6]:
def get_total_table(products_dict: dict):
    df = pd.DataFrame([products_dict])

    dfs = []

    for col in df.columns:
        tmp_df = pd.json_normalize(df[col])
        tmp_df.insert(0, 'product', col)

        dfs.append(tmp_df)


    products = pd.concat(dfs, ignore_index=True).sort_values('total_sum')

    return products

In [7]:
data = read_to_dict()

if data:
    display(get_total_table(data['products']))

    if data['total_sum'] > 100000:
        print(f'Высокие продажи! Общая сумма: {data["total_sum"]:.2f}')


,product,total_amount,total_sum
7,Кофеварка,27,6999.00
4,Пылесос,13,16999.80
6,Микроволновка,18,18999.98
2,Смартфон,10,19999.00
0,Телевизор,5,29999.99
1,Холодильник,2,49999.50
5,Стиральная машина,10,73999.90
3,Ноутбук,4,89999.98


Высокие продажи! Общая сумма: 306997.15


## Task 2. Обработка данных из текстового файла

In [10]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename=RESULTS_PATH+'task2.log',
    level=logging.INFO,
    format='%(levelname)s: %(message)s'
)

In [11]:
def get_avg_score(grades: list):
    return sum(grades) / len(grades)

In [12]:
def read_to_dict(
    filepath: str=SOURCES_PATH + 'students.txt'
):
    '''{
            name: str
            grades: list
        }
    '''

    result = {}

    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            line  = f.readline()
            
            while line:
                data = line.split(',')
                name = data[0]
                grades = data[1:]
                
                grades = list(map(int, grades))

                if result.get(name):
                    result['grades'] += grades
                else:
                    result[name] = grades

                line  = f.readline()

        logging.info(f'Файл {filepath} прочитан успешно')

    except (FileNotFoundError, PermissionError, IOError) as e:
        logging.error(f'Ошибка чтения файла {filepath}: {e}')

    return result

In [13]:
def get_students_with_avg_score_more_than(
    data: dict, 
    target_avg_score: float=3.5,
    write_to_file: bool=True,
    fpath: str=RESULTS_PATH + 'top_students.txt'
):
    top_students = []

    for student, grades in data.items():
        avg_score = get_avg_score(grades)
        if avg_score > target_avg_score:
            top_students.append(student)

    logging.info(f'Общее количество студентов: {len(data)}')

    if write_to_file:
        with open(fpath, 'w', encoding='utf-8') as f:
            f.write('\n'.join(top_students))

    logging.info(f'Информация о студентах успешно записана в {fpath}')
    return top_students

In [14]:
def print_strudents_results(students):
    response = ''
    
    response += 'Результаты студентов:\n'

    for student, grades in students.items():
        response += f'{student}: {grades}\n'

    print(response)

In [15]:
students = read_to_dict()


print_strudents_results(students)

get_students_with_avg_score_more_than(
    students
)

Результаты студентов:
Анна Соколова: [5, 5, 5, 5, 5]
Игорь Лебедев: [2, 3, 3, 2, 4]
Мария Козлова: [4, 5, 4, 5, 5]
Сергей Новиков: [5, 4, 5, 5, 4]
Наталья Морозова: [3, 4, 3, 4, 3]
Алексей Смирнов: [5, 4, 3, 5, 4]
Елена Кузнецова: [4, 4, 5, 5, 5]
Дмитрий Попов: [3, 3, 4, 2, 3]



['Анна Соколова',
 'Мария Козлова',
 'Сергей Новиков',
 'Алексей Смирнов',
 'Елена Кузнецова']

## Task 3. Создание простой базы данных в текстовом файле

In [16]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename=RESULTS_PATH+'task3.log',
    level=logging.INFO,
    format='%(levelname)s: %(message)s'
)

In [17]:
TASKS_FILE = RESULTS_PATH + 'tasks.txt'

CMD_HELP = '''
Команды, которые тебе доступны:
    [help] - Вернуть описание доступных команд
    [add] - Добавить задачу
    [delete] - Удалить конкретную задау
    [get] - Получить полный список задач
    [end] - Выйти из программы
'''

CMD_ERROR = f'''
Такой команды нет!
{CMD_HELP}
'''

In [18]:
def safe_filework(file_manipulation):
    def wrapper(*args, **kwargs):
        try:
            result = file_manipulation(*args, **kwargs)

            logging.info(f'Команда исполнена')
        except (FileNotFoundError, PermissionError, IOError) as e:
            logging.error(f'Ошибка чтения файла {TASKS_FILE}: {e}')
            result = 'С файлом что-то не так'

        return result

    return wrapper


@safe_filework
def read_tasks():
    result = 'Ты пока не сделал ни одной записи'

    if os.path.exists(TASKS_FILE):
        with open(TASKS_FILE, 'r', encoding='utf-8') as f: 
            readed = f.read()
        
        print('Твои таски:')
        result = readed if len(readed) else result

    return result


@safe_filework
def write_task(task):
    task += '\n'
    next_task_no = '1) '
    add_mode = 'w'

    if os.path.exists(TASKS_FILE):
        with open(TASKS_FILE, 'r', encoding='utf-8') as f:
            tasks = f.readlines()

        if len(tasks):
            next_task_no = str(int(tasks[-1].split(')')[0]) + 1) + ') '
            add_mode = 'a'

    task = next_task_no + task

    with open(TASKS_FILE, add_mode, encoding='utf-8') as f: 
        f.write(task)

    return 'Таска успешно добавлена'


@safe_filework
def delete_task(no_task):
    result = 'Таски с таким номером нет!'

    try:
        no_task = int(no_task) - 1

        if os.path.exists(TASKS_FILE):
            with open(TASKS_FILE, 'r', encoding='utf-8') as f:
                lines = f.readlines()

            if no_task >= 0 and no_task < len(lines):
                del lines[no_task]

                lines = [
                    f'{i+1}) {"".join(line.split(") ")[1:])}' for i, line in enumerate(lines)]

                with open(TASKS_FILE, 'w', encoding='utf-8') as f: 
                    f.writelines(lines)

                result = 'Таска успешно удалена'

    except: ...


    return result


In [19]:
def lets_add():
    task = input('Введи таску, которую хочешь добавить: ')
    print(write_task(task))

def lets_delete():
    no_task = input('Введи номер таски, которую хочешь удалить: ')
    print(delete_task(no_task))

def lets_get():
    print(read_tasks())

In [20]:
def run_todoer():

    print('Привет! Давай начнем :)')
    print(CMD_HELP)

    while True:
        cmd = input('Введи команду:')
        logging.info(f'Введена команда: {cmd}')

        if cmd == 'end':
            break
        elif cmd == 'help':
            print(CMD_HELP)
            logging.info(f'Команда исполнена')
        elif cmd == 'add':
            lets_add()
        elif cmd == 'delete':
            lets_delete()
        elif cmd == 'get':
            lets_get()
        else:
            print(CMD_ERROR)
            logging.info(f'Команда не найдена')

    print('Вызови меня снова, как захочешь продолжить :)')
    
    logging.info(f'Конец исполнения программы')


In [21]:
run_todoer()

Привет! Давай начнем :)

Команды, которые тебе доступны:
    [help] - Вернуть описание доступных команд
    [add] - Добавить задачу
    [delete] - Удалить конкретную задау
    [get] - Получить полный список задач
    [end] - Выйти из программы




Команды, которые тебе доступны:
    [help] - Вернуть описание доступных команд
    [add] - Добавить задачу
    [delete] - Удалить конкретную задау
    [get] - Получить полный список задач
    [end] - Выйти из программы

Таска успешно добавлена

Такой команды нет!

Команды, которые тебе доступны:
    [help] - Вернуть описание доступных команд
    [add] - Добавить задачу
    [delete] - Удалить конкретную задау
    [get] - Получить полный список задач
    [end] - Выйти из программы



Такой команды нет!

Команды, которые тебе доступны:
    [help] - Вернуть описание доступных команд
    [add] - Добавить задачу
    [delete] - Удалить конкретную задау
    [get] - Получить полный список задач
    [end] - Выйти из программы


Таска успешно добавлена
Таска успешно добавлена
Твои таски:
1) Lol
2) one
3) two

Таска успешно удалена
Вызови меня снова, как захочешь продолжить :)


# Advanced tasks

## Task 4. Обработка текстового файла и анализ слов

In [22]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename=RESULTS_PATH+'task4.log',
    level=logging.INFO,
    format='%(levelname)s: %(message)s'
)

In [23]:
russian_conjunctions = [
    'и',   
    'а',   
    'но',  
    'или', 
    'да',  
    'тоже',
    'если',
    'потому',
    'что',
    'так как',
    'хотя',   
    'когда',  
    'где',    
    'чем',    
    'как',    
    'чтобы',  
    'поэтому',
    'то',
    'есть' 
]

russian_prepositions = [
    "в",
    "на",
    "о",
    "по",
    "с",
    "без",
    "из",
    "к",
    "от",
    "для",
    "между",
    "над",
    "под",
    "у",
    "за",
    "перед",
    "вдоль",
    "вместо",
    "вне",
    "внутри",
    "внутрь"
]

STOP_WORDS = russian_conjunctions + russian_prepositions


In [24]:
def safe_filework(file_manipulation):
    def wrapper(fpath, *args, **kwargs):
        try:
            result = file_manipulation(fpath, *args, **kwargs)

            logging.info(f'Файл {fpath} открыт успешно')
        except (FileNotFoundError, PermissionError, IOError) as e:
            logging.error(f'Ошибка открытия файла {fpath}: {e}')
            result = None

        return result

    return wrapper


@safe_filework
def read_text(fpath):
    with open(fpath, 'r', encoding='utf-8') as f:
        result  = f.read()

    return result


@safe_filework
def dict_to_json(
    fpath: str,
    target_dict: dict
):
    with open(fpath, 'w', encoding='utf-8') as f:
        json.dump(
            target_dict, 
            f, 
            ensure_ascii=False, 
            indent=4
        )

    return True

In [25]:
def count_words(text: str):
    text = re.sub(
        r'[^a-zA-Zа-яА-Я\s]', '', text
    ).lower()

    words_bag = [
        word for word in re.split(r'[\s\n]', text)
        if word not in STOP_WORDS and len(word) > 0
    ]

    unique_words = set(words_bag)
    logging.info(f'Количество уникальных слов: {len(unique_words)}')

    words_amout = {word: 0 for word in unique_words}
    total_amount = 0

    for word in words_bag:
        words_amout[word] += 1
        total_amount += 1

    logging.info(f'Общее количество слов: {total_amount}')

    return words_amout



In [26]:
text = read_text(SOURCES_PATH + 'text.txt')

words = count_words(text)

display(words)

dict_to_json(RESULTS_PATH + 'word_count.json', words)

{'конечно': 1,
 'радости': 2,
 'веселятся': 1,
 'иногда': 1,
 'атмосферу': 1,
 'другие': 1,
 'счастлива': 1,
 'любопытная': 1,
 'зайца': 1,
 'может': 1,
 'друзей': 1,
 'жизнь': 1,
 'ее': 1,
 'любимая': 1,
 'приключений': 1,
 'потом': 1,
 'создают': 1,
 'вечер': 1,
 'звуки': 1,
 'поиграть': 1,
 'вернуться': 1,
 'лесу': 4,
 'играют': 1,
 'лес': 2,
 'белки': 1,
 'эти': 1,
 'живет': 1,
 'птицу': 1,
 'трели': 1,
 'каждое': 1,
 'это': 1,
 'орехи': 4,
 'интересного': 1,
 'красивые': 1,
 'исследует': 1,
 'всегда': 1,
 'долгого': 1,
 'маленькая': 1,
 'полон': 1,
 'много': 1,
 'местах': 1,
 'своих': 1,
 'дня': 1,
 'животные': 1,
 'собирать': 1,
 'возвращается': 1,
 'они': 2,
 'шорох': 1,
 'она': 2,
 'звуков': 1,
 'образом': 1,
 'яркие': 1,
 'таким': 1,
 'встречает': 1,
 'после': 1,
 'друзьями': 1,
 'прячет': 1,
 'ведь': 1,
 'домой': 1,
 'растут': 1,
 'птиц': 1,
 'ними': 1,
 'собирает': 1,
 'деревья': 1,
 'эта': 1,
 'цветы': 1,
 'листьев': 1,
 'ягоды': 1,
 'отдохнуть': 1,
 'спокойствия': 1,
 'оче

True

## Task 5. Работа с API и обработка данных

In [27]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename=RESULTS_PATH+'task5.log',
    level=logging.INFO,
    format='%(levelname)s: %(message)s'
)

In [28]:
def save_results(
    data: dict,
    fpath: str=RESULTS_PATH + 'weather.json'
):
    try:
        with open(fpath, 'w', encoding='utf-8') as f:
            json.dump(
                data,
                f,
                ensure_ascii=False,
                indent=4
            )
        logging.info(f'Данные успешно сохранены в файл {fpath}')
    except (FileNotFoundError, PermissionError, IOError) as e:
        logging.error(f'Ошибка сохранения данных в файл {fpath}: {e}')
        


In [29]:
def get_weather(city: str):
    weather_api_url = f'https://api.openweathermap.org/data/2.5/weather'
    params = {
        'q': city,
        'appid': '1bf1c2261b9c8eece595c9f926a72e35',
        'units': 'metric'
    }

    weather = {}

    try:
        response = requests.get(
            weather_api_url, 
            params=params,
            timeout=5
        )

        if response.status_code == 200:
            all_about_weather = response.json()
            
            save_results(all_about_weather)

            weather['description'] = all_about_weather['weather']
            if weather['description']: 
                weather['description'] = weather['description'][0].get('description')

            weather['temp'] = all_about_weather['main'].get('temp')
            weather['feels_like'] = all_about_weather['main'].get('feels_like')
            weather['wind_speed'] = all_about_weather['wind'].get('speed')
            
            logging.info('Запрос выполнен успешно')
        elif response.status_code == 404:
            logging.error(f'Ошибка ответа: введен некорректный город {city}')
        else:
            logging.error(f'Ошибка ответа: status code: {response.status_code}. См. FAQ https://openweathermap.org/faq')
    
    except requests.exceptions.Timeout:
        logging.error('Сервер не отвечает: Time out')
    
    except:
        logging.error('Ошибка запроса')

    return weather


In [30]:
def print_weather(weather: dict, city: str):
    print_string = \
    f'***** Текущая погода в славном городе {city} *****\n'\
    f'\t- Описание погодных условий: {weather["description"]}\n'\
    f'\t- Температура: {weather["temp"]}°\n'\
    f'\t- Ощущается, как: {weather["feels_like"]}°\n'\
    f'\t- Скорость ветра: {weather["wind_speed"]} м/сек\n'

    print(print_string)

In [31]:
cities = ['Moscow', 'Mosc', 'London', 'Gudauta']

for city in cities:
    weather = get_weather(city)

    if weather:
        print_weather(weather, city)
    else:
        print(f'Ой ой. Что-то не так. Проверь файл {RESULTS_PATH + "task5.log"}')

    print('\n')

***** Текущая погода в славном городе Moscow *****
	- Описание погодных условий: overcast clouds
	- Температура: 8.87°
	- Ощущается, как: 8.87°
	- Скорость ветра: 0.9 м/сек



Ой ой. Что-то не так. Проверь файл ./tmp/task5.log


***** Текущая погода в славном городе London *****
	- Описание погодных условий: overcast clouds
	- Температура: 14.91°
	- Ощущается, как: 14.23°
	- Скорость ветра: 1.54 м/сек



***** Текущая погода в славном городе Gudauta *****
	- Описание погодных условий: few clouds
	- Температура: 14.06°
	- Ощущается, как: 13.17°
	- Скорость ветра: 1.5 м/сек





## Task 8. Словарь синонимов

In [32]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename=RESULTS_PATH+'task8.log',
    level=logging.INFO,
    format='%(levelname)s: %(message)s'
)

In [33]:
def read_to_dict(
    filepath: str=SOURCES_PATH + 'synonyms.json'
):
    '''Expects
        word: [synonym, ...]
    '''
    results = {}

    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            results = json.load(f)

        results = {
            key.lower(): 
            [synonim.lower() for synonim in val] 
            for key, val in results.items()
        }

        logging.info(f'Файл {filepath} прочитан успешно')

    except (FileNotFoundError, PermissionError, IOError) as e:
        logging.error(f'Ошибка чтения файла {filepath}: {e}')

    except Exception as e:
        logging.error(f'Некорректный json {filepath}: {e}')

    return results

In [34]:
SYNONIMS = read_to_dict()

SYNONIMS

{'счастливый': ['радостный', 'веселый', 'довольный', 'благополучный'],
 'грустный': ['печальный', 'унылый', 'тоскливый', 'депрессивный'],
 'быстрый': ['скорый', 'стремительный', 'мгновенный', 'оперативный'],
 'медленный': ['постепенный', 'неторопливый', 'вялый', 'ленивый'],
 'умный': ['разумный', 'интеллектуальный', 'умелый', 'сообразительный'],
 'злой': ['агрессивный', 'сердитый', 'враждебный', 'недоброжелательный'],
 'красивый': ['привлекательный', 'симпатичный', 'очаровательный', 'гладкий'],
 'сильный': ['мощный', 'крепкий', 'выносливый', 'твердый'],
 'смешной': ['юмористический', 'комичный', 'забавный', 'курьезный'],
 'трудный': ['сложный', 'непростой', 'затруднительный', 'усложненный'],
 'новый': ['современный', 'новейший', 'инновационный', 'недавний'],
 'старый': ['древний', 'ветхий', 'устаревший', 'прошлый'],
 'умиротворенный': ['спокойный', 'тихий', 'безмятежный', 'уравновешенный'],
 'активный': ['энергичный', 'живой', 'подвижный', 'деятельный'],
 'пассивный': ['неактивный', 'и

In [35]:
def get_synonyms(word: str) -> str:
    if SYNONIMS.get(raw_word := word.lower()):
        synonim_words = f'Вот список синонимов для слова "{raw_word}":\n'
        synonim_words += '\n'.join(
            f'\t{i+1}. {s_word}' for i, s_word in enumerate(SYNONIMS[raw_word])
        )
        logging.info(f'Слово {word} найдено в словаре')
    else:
        logging.info(f'Слово {word} не найдено в словаре')
        synonim_words = f'Такое слово "{raw_word}" не найдено'
    
    synonim_words += '\n*** Для выхода введи "!end!" ***\n'

    return synonim_words

In [36]:
print('Привет, я могу вернуть синонимы к слову, если оно есть у меня в словаре :)')

while (word := input('Введи слово, для которого хочешь получить синонимы: ')) != '!end!':
    logging.info(f'Пользователь ввел: {word}')
    print(get_synonyms(word))


print('Запусти меня снова, если захочешь продолжить :)')
logging.info('Программа завершила работу')
    

Привет, я могу вернуть синонимы к слову, если оно есть у меня в словаре :)
Вот список синонимов для слова "красивый":
	1. привлекательный
	2. симпатичный
	3. очаровательный
	4. гладкий
*** Для выхода введи "!end!" ***

Такое слово "нет" не найдено
*** Для выхода введи "!end!" ***

Запусти меня снова, если захочешь продолжить :)
